In [2]:
import pandas as pd
import numpy as np
from scipy import spatial
import math
import polars as pl
import csv
import random
import tqdm

In [3]:
weapons = ["bomb", "rifle", "arrow"]

In [4]:
arr_random_input = np.random.default_rng().uniform(low=-100,high=100,size=[10000000,2])
arr_random_query = np.random.default_rng().uniform(low=-100,high=100,size=[1000,2])
df = pd.DataFrame(arr_random_input, columns=["x","y"])
query_df = pd.DataFrame(arr_random_query, columns=["x","y"])
df["weapon"] = "a"
df["weapon"]=df.weapon.apply(lambda x:random.choice(weapons))
df.to_parquet("notebook_results/input.parquet")
query_df.to_parquet("notebook_results/query.parquet")

### This seems to work, have to create the algorithm for it

In [5]:
out = []
for idx, row in query_df.iterrows():

    diffs = (df[["x", "y"]] - row).pipe(lambda df: df**2).sum(axis=1)
    out.append({"dist": math.sqrt(diffs.min()), "weapon": df.loc[diffs.argmin(), "weapon"]})
pd.DataFrame(out).to_csv("notebook_results/out.csv", index=False)

KeyboardInterrupt: 

In [46]:
%%timeit
out = []
for idx, row in query_df.iterrows():

    diffs = (df[["x", "y"]] - row).pipe(lambda df: df**2).sum(axis=1)
    out.append({"dist": math.sqrt(diffs.min()), "weapon": df.loc[diffs.argmin(), "weapon"]})
pd.DataFrame(out).to_csv("notebook_results/out.csv", index=False)

2.24 s ± 249 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# This is 20x better than the original
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
#pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)


In [5]:
%%timeit
tree = spatial.KDTree(df[["x","y"]])
out = []
for idx, row in query_df.iterrows():
    out.append({"dist": tree.query(row)[0], "weapon": df.loc[tree.query(row)[1], "weapon"]})
#pd.DataFrame(out).to_csv("notebook_results/out_tree.csv", index=False)

12.7 s ± 1.14 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


try with polars

In [63]:
query_df_p = pl.read_parquet("notebook_results/query.parquet")
df_p = pl.read_parquet("notebook_results/input.parquet")

In [28]:
#df_p = df_p.with_row_index("rownum",0)

In [66]:
tree = spatial.KDTree(df_p.select("x","y"))

In [31]:
tree.query(query_df_p[1])[1]

array([7096922], dtype=int64)

In [51]:
for row in query_df_p.iter_rows():
    asd = df_p[int(tree.query(row)[1])]["weapon"]

In [32]:
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], "weapon": tree.query(row)[1]})

In [61]:
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})

In [62]:
out

[{'dist': 0.039278713604762906, 'weapon': 'arrow'},
 {'dist': 0.01255250093232448, 'weapon': 'arrow'},
 {'dist': 0.0454423380376236, 'weapon': 'bomb'},
 {'dist': 0.020370120113962855, 'weapon': 'bomb'},
 {'dist': 0.06377640602534053, 'weapon': 'arrow'},
 {'dist': 0.02119076022437295, 'weapon': 'arrow'},
 {'dist': 0.039854343687682675, 'weapon': 'rifle'},
 {'dist': 0.034926648826628916, 'weapon': 'bomb'},
 {'dist': 0.01371150166194409, 'weapon': 'bomb'},
 {'dist': 0.020640866877991323, 'weapon': 'rifle'},
 {'dist': 0.022115639139974424, 'weapon': 'bomb'},
 {'dist': 0.06262065025342188, 'weapon': 'bomb'},
 {'dist': 0.04072283216971242, 'weapon': 'arrow'},
 {'dist': 0.010458516002231388, 'weapon': 'rifle'},
 {'dist': 0.008954094804945337, 'weapon': 'rifle'},
 {'dist': 0.03429964218136761, 'weapon': 'bomb'},
 {'dist': 0.021076181042809665, 'weapon': 'bomb'},
 {'dist': 0.03917367220884501, 'weapon': 'bomb'},
 {'dist': 0.04537270740664906, 'weapon': 'rifle'},
 {'dist': 0.0351381444902152, 'w